In [2]:
#functions for board
def print_board(board):
    print("|-------------|")
    print("| Tic Tac Toe |")
    print("|-------------|")
    print("|             |")
    print("|    " + board[0][0] + " " + board[0][1] + " " + board[0][2] + "    |")
    print("|    " + board[1][0] + " " + board[1][1] + " " + board[1][2] + "    |")
    print("|    " + board[2][0] + " " + board[2][1] + " " + board[2][2] + "    |")
    print("|             |")
    print("|-------------|")
    print()


def select_space(board, move, turn):
    if move not in range(1,10):
        return False
    row = int((move-1)/3)
    col = (move-1)%3
    if board[row][col] != "X" and board[row][col] != "O":
        board[row][col] = turn
        return True
    else:
        return False

def available_moves(board):
    moves = []
    for row in board:
        for col in row:
            if col != "X" and col != "O":
                moves.append(int(col))
    return moves

def has_won(board, player):
    for row in board:
        if row.count(player) == 3:
            return True
    for i in range(3):
        if board[0][i] == player and board[1][i] == player and board[2][i] == player:
            return True
    if board[0][0] == player and board[1][1] == player and board[2][2] == player:
        return True
    if board[0][2] == player and board[1][1] == player and board[2][0] == player:
        return True
    return False

In [3]:
#testing above funcs
my_board = [
    ["1", "2", "X"],
    ["4", "5", "6"],
    ["7", "8", "9"]
]
print_board(my_board)
select_space(my_board,5,"O")
select_space(my_board,2,"X")
select_space(my_board,8,"O")
select_space(my_board,1,"X")
print_board(my_board)
print(available_moves(my_board))
print(has_won(my_board,"X"))
print(has_won(my_board,"O"))

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    1 2 X    |
|    4 5 6    |
|    7 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X X    |
|    4 O 6    |
|    7 O 9    |
|             |
|-------------|

[4, 6, 7, 9]
True
False


#### Detecting Tic-Tac-Toe Leaves
**An essential step in the minimax function is evaluating the strength of a leaf.** 
- One potential evaluation function: a leaf where player "X" wins evaluates to a 1, a leaf where player "O" wins evaluates to a -1, and a leaf that is a tie evaluates to 0
- to detect whether a board is a leaf — need to know if the game is over.


In [4]:
start_board = [
    ["1", "2", "3"],
    ["4", "5", "6"],
    ["7", "8", "9"]
]

x_won = [
    ["X", "O", "3"],
    ["4", "X", "O"],
    ["7", "8", "X"]
]

o_won = [
    ["O", "X", "3"],
    ["O", "X", "X"],
    ["O", "8", "9"]
]

tie = [
    ["X", "X", "O"],
    ["O", "O", "X"],
    ["X", "O", "X"]
]

In [5]:
def game_is_over(board ):
  if len(available_moves(board))==0 or has_won(board, "X") or has_won(board, "O"):
    return True
  else: return False

print(game_is_over(start_board))
print(game_is_over(x_won))
print(game_is_over(o_won))
print(game_is_over(tie)) 

False
True
True
True


In [6]:
def evaluate_board(board):
  if has_won(board, "X"):
    return 1
  elif has_won(board, "O"):
    return -1
  elif len(available_moves(board))==0:
    return 0
  
if game_is_over(start_board):
  print(evaluate_board(start_board))
if game_is_over(x_won):
  print(evaluate_board(x_won))
if game_is_over(o_won):
  print(evaluate_board(o_won))
if game_is_over(tie):
  print(evaluate_board(tie))

1
-1
0


#### Evaluating Leaves
Let's imagine a situation where you're playing as the "X" player in a game of Tic-Tac-Toe and the game is almost over. The game board isn't a leaf but it's close. You have three possible moves. All three moves will immediately end the game — each of those future boards will be leaves.

Let's say picking move A will result in you winning and moves B and C will each result in a tie. You'd clearly pick move A.

By picking move A, you've picked the move that led to the board with the highest value. You were picking between a 1 (an "X" win) or two 0s (the moves that would lead to a tie). Because you picked the move with the highest value, we can say that "X" is the maximizing player.

Let's say you were playing a the "O" player under the same circumstances. Picking move A would somehow immediately lead to "X" winning, while moves B and C would lead to a tie. You'd pick one of the boards that would lead to a tie. "O" is the minimizing player. You would love to pick a board with the value of -1 (an "O" win), but unfortunately, that board doesn't exist. You'll have to settle with picking a board with the value of 0. At least you prevent "X" from winning.

**Copying Boards**
One of the central ideas behind the minimax algorithm is the idea of exploring future hypothetical board states.
- Don't want to actually make move on the board.Make a copy of the board and make the move on that one.

In [7]:
from copy import deepcopy
my_board = [
    ["1", "2", "X"],
    ["4", "5", "6"],
    ["7", "8", "9"]
]
new_board  = deepcopy(my_board)

The result of **Minimax** function will be the "value" of the best possible move
- returns a 1, that means a move exists that guarantees that "X" will win
- returns a -1, that means that there's nothing that "X" can do to prevent "O" from winning. 
- returns a 0, then the best "X" can do is force a tie (assuming "O" doesn't make a mistake).

In [8]:
def minimax(input_board, is_maximizing):
  # Base case - the game is over, so we return the value of the board
  if game_is_over(input_board):
    return [evaluate_board(input_board), ""]
  #--------------------if the node isn't a leaf--------------------#
  best_move = ""
  if is_maximizing == True:
    best_value = -float("Inf")
    symbol = "X"
  else:
    best_value = float("Inf")
    symbol = "O"
 #loop through all of the possible moves of input_board and make the move!
  for move in available_moves(input_board):
    new_board = deepcopy(input_board)
    select_space(new_board, move, symbol)
    hypothetical_value = minimax(new_board, not is_maximizing)[0]
  #know whether the value of that board is better than our current best_value and note best move
    if is_maximizing == True and hypothetical_value > best_value:
      best_value = hypothetical_value
      best_move = move
    if is_maximizing == False and hypothetical_value < best_value:
      best_value = hypothetical_value
      best_move = move
  return [best_value, best_move]

**Play a Game**

In [9]:
my_board = [
    ["1", "2", "3"],
    ["4", "5", "6"],
    ["7", "8", "9"]
]

while not game_is_over(my_board):
  select_space(my_board, minimax(my_board, True)[1], "X")
  print_board(my_board)
  if not game_is_over(my_board):
    select_space(my_board, minimax(my_board, False)[1], "O")
    print_board(my_board)  

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X 2 3    |
|    4 5 6    |
|    7 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X 2 3    |
|    4 O 6    |
|    7 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X 3    |
|    4 O 6    |
|    7 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X O    |
|    4 O 6    |
|    7 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X O    |
|    4 O 6    |
|    X 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X O    |
|    O O 6    |
|    X 8 9    |
|             |
|-------------|

|-------------|
| Tic Tac Toe |
|-------------|
|             |
|    X X O    |
|    O O X    |
|    X 8 9    |
|             |
|-